In [21]:
import duckdb


In [22]:
file_train = 'data/NMAD_with_embeddings_track.parquet'

In [23]:
con = duckdb.connect(file_train)

In [25]:
con.execute("select * from NMAD_with_embeddings_track limit 10").df()

,rgt,track,spot,height,geoid_height,orthometric_height,h_alos_dem,delta_alos_dem,abs_delta_alos_dem,h_aster_dem,...,emb_056,emb_057,emb_058,emb_059,emb_060,emb_061,emb_062,emb_063,emb_064,emb_all
0,556.0,1.0,5.0,1069.885864,36.636,1033.249864,1054.0,20.750136,20.750136,1046.0,...,0.192910,-0.130165,0.292872,-0.267958,-0.000554,0.093564,0.032541,0.062991,0.041584,"[0.1599999964237213, -0.04484429210424423, 0.0..."
1,556.0,1.0,5.0,1069.885864,36.636,1033.249864,1054.0,20.750136,20.750136,1046.0,...,0.192910,-0.130165,0.292872,-0.267958,-0.000554,0.093564,0.032541,0.062991,0.041584,"[0.1599999964237213, -0.04484429210424423, 0.0..."
2,556.0,1.0,5.0,1069.440063,36.636,1032.804063,1044.0,11.195937,11.195937,1046.0,...,0.199862,-0.124567,0.292872,-0.267958,0.004983,0.103406,0.024606,0.079723,0.062991,"[0.1599999964237213, -0.06299115717411041, -0...."
3,556.0,1.0,5.0,1069.440063,36.636,1032.804063,1044.0,11.195937,11.195937,1046.0,...,0.199862,-0.124567,0.292872,-0.267958,0.004983,0.103406,0.024606,0.079723,0.062991,"[0.1599999964237213, -0.06299115717411041, -0...."
4,556.0,1.0,5.0,1070.128174,36.636,1033.492174,1044.0,10.507826,10.507826,1046.0,...,0.199862,-0.124567,0.292872,-0.267958,0.004983,0.103406,0.024606,0.079723,0.062991,"[0.1599999964237213, -0.06299115717411041, -0...."
5,556.0,1.0,5.0,1070.128174,36.636,1033.492174,1044.0,10.507826,10.507826,1046.0,...,0.199862,-0.124567,0.292872,-0.267958,0.004983,0.103406,0.024606,0.079723,0.062991,"[0.1599999964237213, -0.06299115717411041, -0...."
6,556.0,1.0,5.0,1066.541626,36.636,1029.905626,1044.0,14.094374,14.094374,1046.0,...,0.179377,-0.130165,0.292872,-0.267958,0.000984,0.113741,0.029773,0.088827,0.084214,"[0.15378700196743011, -0.06698962301015854, -0..."
7,556.0,1.0,5.0,1066.242065,36.636,1029.606065,1044.0,14.393935,14.393935,1046.0,...,0.179377,-0.130165,0.292872,-0.267958,0.000984,0.113741,0.029773,0.088827,0.084214,"[0.15378700196743011, -0.06698962301015854, -0..."
8,556.0,1.0,5.0,1066.541626,36.636,1029.905626,1044.0,14.094374,14.094374,1046.0,...,0.179377,-0.130165,0.292872,-0.267958,0.000984,0.113741,0.029773,0.088827,0.084214,"[0.15378700196743011, -0.06698962301015854, -0..."
9,556.0,1.0,5.0,1066.242065,36.636,1029.606065,1044.0,14.393935,14.393935,1046.0,...,0.179377,-0.130165,0.292872,-0.267958,0.000984,0.113741,0.029773,0.088827,0.084214,"[0.15378700196743011, -0.06698962301015854, -0..."


In [26]:
shema = con.execute("DESCRIBE SELECT * FROM NMAD_with_embeddings").fetchdf()
print([x for x in shema.column_name])

CatalogException: Catalog Error: Table with name NMAD_with_embeddings does not exist!
Did you mean "nmad_with_embeddings_track"?

LINE 1: DESCRIBE SELECT * FROM NMAD_with_embeddings
                               ^

In [7]:
con.sql("SELECT quantile_cont(nmad_fab, 0.33) AS q33, quantile_cont(nmad_fab, 0.66) AS q66 FROM NMAD_with_embeddings;")

┌────────────────────┬────────────────────┐
│        q33         │        q66         │
│       double       │       double       │
├────────────────────┼────────────────────┤
│ 1.7339919146483445 │ 3.2349415944317137 │
└────────────────────┴────────────────────┘

In [28]:
import duckdb
import pandas as pd
file_train = 'data/NMAD_with_embeddings_track.parquet'

con = duckdb.connect(file_train)

# 1) витягаємо квантілі
q33, q66 = con.execute("""
    SELECT quantile_cont(nmad_fab, 0.33), quantile_cont(nmad_fab, 0.66)
    FROM NMAD_with_embeddings_track
""").fetchone()

# 2) створюємо нову таблицю з класами
con.execute(f"""
    CREATE OR REPLACE TABLE NMAD_with_embeddings_cls AS
    SELECT *,
        CASE
            WHEN nmad_fab <= {q33} THEN 0
            WHEN nmad_fab <= {q66} THEN 1
            ELSE 2
        END AS cls_nmad_fab
    FROM NMAD_with_embeddings_track
""")

# 3) зберігаємо в parquet
con.execute("""
    COPY NMAD_with_embeddings_cls TO 'NMAD_with_embeddings_cls.parquet' (FORMAT PARQUET)
""")


In [29]:
file_train_cls = 'data/NMAD_with_embeddings_cls.parquet'
con = duckdb.connect(file_train_cls)
con.execute("select * from NMAD_with_embeddings_cls limit 10").df()

,rgt,track,spot,height,geoid_height,orthometric_height,h_alos_dem,delta_alos_dem,abs_delta_alos_dem,h_aster_dem,...,emb_057,emb_058,emb_059,emb_060,emb_061,emb_062,emb_063,emb_064,emb_all,cls_nmad_fab
0,556.0,1.0,5.0,1069.885864,36.636,1033.249864,1054.0,20.750136,20.750136,1046.0,...,-0.130165,0.292872,-0.267958,-0.000554,0.093564,0.032541,0.062991,0.041584,"[0.1599999964237213, -0.04484429210424423, 0.0...",0
1,556.0,1.0,5.0,1069.885864,36.636,1033.249864,1054.0,20.750136,20.750136,1046.0,...,-0.130165,0.292872,-0.267958,-0.000554,0.093564,0.032541,0.062991,0.041584,"[0.1599999964237213, -0.04484429210424423, 0.0...",0
2,556.0,1.0,5.0,1069.440063,36.636,1032.804063,1044.0,11.195937,11.195937,1046.0,...,-0.124567,0.292872,-0.267958,0.004983,0.103406,0.024606,0.079723,0.062991,"[0.1599999964237213, -0.06299115717411041, -0....",1
3,556.0,1.0,5.0,1069.440063,36.636,1032.804063,1044.0,11.195937,11.195937,1046.0,...,-0.124567,0.292872,-0.267958,0.004983,0.103406,0.024606,0.079723,0.062991,"[0.1599999964237213, -0.06299115717411041, -0....",1
4,556.0,1.0,5.0,1070.128174,36.636,1033.492174,1044.0,10.507826,10.507826,1046.0,...,-0.124567,0.292872,-0.267958,0.004983,0.103406,0.024606,0.079723,0.062991,"[0.1599999964237213, -0.06299115717411041, -0....",0
5,556.0,1.0,5.0,1070.128174,36.636,1033.492174,1044.0,10.507826,10.507826,1046.0,...,-0.124567,0.292872,-0.267958,0.004983,0.103406,0.024606,0.079723,0.062991,"[0.1599999964237213, -0.06299115717411041, -0....",0
6,556.0,1.0,5.0,1066.541626,36.636,1029.905626,1044.0,14.094374,14.094374,1046.0,...,-0.130165,0.292872,-0.267958,0.000984,0.113741,0.029773,0.088827,0.084214,"[0.15378700196743011, -0.06698962301015854, -0...",2
7,556.0,1.0,5.0,1066.242065,36.636,1029.606065,1044.0,14.393935,14.393935,1046.0,...,-0.130165,0.292872,-0.267958,0.000984,0.113741,0.029773,0.088827,0.084214,"[0.15378700196743011, -0.06698962301015854, -0...",2
8,556.0,1.0,5.0,1066.541626,36.636,1029.905626,1044.0,14.094374,14.094374,1046.0,...,-0.130165,0.292872,-0.267958,0.000984,0.113741,0.029773,0.088827,0.084214,"[0.15378700196743011, -0.06698962301015854, -0...",2
9,556.0,1.0,5.0,1066.242065,36.636,1029.606065,1044.0,14.393935,14.393935,1046.0,...,-0.130165,0.292872,-0.267958,0.000984,0.113741,0.029773,0.088827,0.084214,"[0.15378700196743011, -0.06698962301015854, -0...",2


In [20]:
shema_cat = con.execute("DESCRIBE SELECT * FROM NMAD_with_embeddings_cls").fetchdf()
print([x for x in shema_cat.column_name])

['height', 'geoid_height', 'orthometric_height', 'h_alos_dem', 'delta_alos_dem', 'abs_delta_alos_dem', 'h_aster_dem', 'delta_aster_dem', 'abs_delta_aster_dem', 'h_copernicus_dem', 'delta_copernicus_dem', 'abs_delta_copernicus_dem', 'h_fab_dem', 'delta_fab_dem', 'abs_delta_fab_dem', 'h_nasa_dem', 'delta_nasa_dem', 'abs_delta_nasa_dem', 'h_srtm_dem', 'delta_srtm_dem', 'abs_delta_srtm_dem', 'h_tan_dem', 'delta_tan_dem', 'abs_delta_tan_dem', 'lulc_class', 'lulc_name', 'alos_dem_slope', 'aster_dem_slope', 'copernicus_dem_slope', 'fab_dem_slope', 'nasa_dem_slope', 'srtm_dem_slope', 'tan_dem_slope', 'alos_dem_geomorphon', 'alos_dem_landform', 'aster_dem_geomorphon', 'aster_dem_landform', 'copernicus_dem_geomorphon', 'copernicus_dem_landform', 'fab_dem_geomorphon', 'fab_dem_landform', 'nasa_dem_geomorphon', 'nasa_dem_landform', 'srtm_dem_geomorphon', 'srtm_dem_landform', 'tan_dem_geomorphon', 'tan_dem_landform', 'alos_dem_2000', 'aster_dem_2000', 'copernicus_dem_2000', 'fab_dem_2000', 'nasa_de

In [31]:
df = con.execute("""
  SELECT track, rgt, spot, cls_nmad_fab, x_merc, y_merc
  FROM NMAD_with_embeddings_cls
""").fetchdf()

# Розмір груп (rgt,track,spot)
g = df.groupby(["rgt","track","spot"], as_index=False).size().rename(columns={"size":"n"})
g = g.sort_values("n", ascending=False)
print(g.head(10), g.tail(10))

      rgt  track  spot       n
10  396.0    3.0   1.0  388584
8   396.0    2.0   3.0  298258
26  998.0    3.0   1.0  200506
24  998.0    2.0   3.0  187001
6   396.0    1.0   5.0  148075
12  396.0    3.0   5.0  141245
11  396.0    3.0   2.0   79113
9   396.0    2.0   4.0   73538
22  998.0    1.0   5.0   66001
20  998.0    1.0   1.0   62564        rgt  track  spot     n
33  1341.0    3.0   2.0  4177
16   556.0    1.0   5.0  3667
31  1341.0    2.0   4.0  3292
2     53.0    3.0   5.0  2455
1     53.0    3.0   2.0  2209
15   556.0    1.0   2.0  2177
18   556.0    2.0   3.0  1148
17   556.0    1.0   6.0   870
3     53.0    3.0   6.0   403
19   556.0    2.0   4.0   179


In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

# 1) Таргет
y = df["cls_nmad_fab"].astype(int)

# 2) Групи = RGT×track (і ДАЄМО НАЗВУ!)
df["group_id"] = df["rgt"].astype(int).astype(str) + "_" + df["track"].astype(int).astype(str)
groups = df["group_id"]

# 3) Розмір груп і фільтр "малих" для тесту
gsize = df.groupby("group_id").size().rename("n")
df = df.join(gsize, on="group_id")

MIN_TEST_N = 3000
is_big = df["n"] >= MIN_TEST_N

# 4) Формуємо індекси лише з великих груп для sgkf
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

big_idx = np.where(is_big.values)[0]
y_big   = y.iloc[big_idx]
grp_big = groups.iloc[big_idx]

folds = []
for fold, (tr_big, te_big) in enumerate(sgkf.split(big_idx, y_big, groups=grp_big), 1):
    tr_idx = big_idx[tr_big]
    te_idx = big_idx[te_big]
    # додаємо всі "малі" групи в train
    tr_idx = np.concatenate([tr_idx, np.where(~is_big.values)[0]])

    folds.append((np.sort(tr_idx), np.sort(te_idx)))

    # інформаційний принт
    print(
        f"Fold {fold}: "
        f"train groups = {df.iloc[tr_idx]['group_id'].nunique()} | "
        f"test groups = {df.iloc[te_idx]['group_id'].nunique()}"
    )

# Додаткові перевірки
for k, (tr, te) in enumerate(folds, 1):
    # 1) групи не перетинаються
    assert set(df.iloc[tr]["group_id"]).isdisjoint(set(df.iloc[te]["group_id"]))
    # 2) баланс класів
    print(f"\nFold {k}")
    print("train class dist:", y.iloc[tr].value_counts(normalize=True).sort_index().round(3).to_dict())
    print("test  class dist:", y.iloc[te].value_counts(normalize=True).sort_index().round(3).to_dict())


ValueError: columns overlap but no suffix specified: Index(['n'], dtype='object')